# Lab27 CNN with Custom Dataset

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)

In [11]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers = 2)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29,120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [8]:
# Testing
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [9]:
optimizer = optim.Adam(net.parameters(), lr=0.00001)
loss_func = nn.CrossEntropyLoss().to(device)

In [13]:
total_batch = len(data_loader)

epochs = 5
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

[Epoch: 1] cost = 0.5883588790893555
[Epoch: 2] cost = 0.5076980590820312
[Epoch: 3] cost = 0.4151124060153961
[Epoch: 4] cost = 0.32879504561424255
[Epoch: 5] cost = 0.2545679807662964
Learning Finished!


In [15]:
torch.save(net.state_dict(), "./model/model.pth")

In [16]:
new_net = CNN().to(device)

In [17]:
new_net.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [18]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)

In [19]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [20]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        correct_prediction = torch.argmax(prediction,1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
